In [82]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_video_transcript(url):
    video_id = url.replace('https://www.youtube.com/watch?v=', '')
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    clean_transcript = ''
    for x in transcript:
        sentence = x['text']
        clean_transcript += f' {sentence}'
    return transcript, clean_transcript

url = 'https://www.youtube.com/watch?v=1DEtdr_oc9s'

text, clean_transcript = get_video_transcript(url)


print(url)
print(text, clean_transcript)
print(type(text))

https://www.youtube.com/watch?v=1DEtdr_oc9s
[{'text': 'hi everyone so we are we are here to', 'start': 2.44, 'duration': 8.44}, {'text': 'present the pp for the logon so without', 'start': 5.68, 'duration': 7.72}, {'text': "any further Ado let's start with the", 'start': 10.88, 'duration': 5.239}, {'text': 'presentation so the problem statement a', 'start': 13.4, 'duration': 4.84}, {'text': 'problem statement is a self-learning a', 'start': 16.119, 'duration': 5.24}, {'text': 'power PDF to data converter uh our team', 'start': 18.24, 'duration': 5.44}, {'text': 'name is team conference the members of', 'start': 21.359, 'duration': 4.32}, {'text': 'our team are M', 'start': 23.68, 'duration': 6.439}, {'text': 'myself ban and PR we are from vishak', 'start': 25.679, 'duration': 6.88}, {'text': 'Institute of Technology', 'start': 30.119, 'duration': 6.041}, {'text': 'Pune so the idea approach and details so', 'start': 32.559, 'duration': 6.281}, {'text': 'a solution a proposed solution ut

In [83]:
from datetime import datetime

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    return f"{int(minutes):02d}:{seconds:.2f}"

def time_to_seconds(time_str):
    minutes, seconds = map(float, time_str.split(':'))
    return minutes * 60 + seconds

combined_segments = []
group_size = 4 

for i in range(0, len(text), group_size):
    group_text_segments = [item['text'] for item in text[i:i+group_size]]
    combined_text_segment = ' '.join(group_text_segments)

    start_times = [item['start'] for item in text[i:i+group_size]]
    min_start_time = min(start_times)

    combined_segments.append({
        'text': combined_text_segment,
        'start': format_time(round(min_start_time, 2)),
        'end': format_time(round(min_start_time, 2))
    })

for i in range(len(combined_segments) - 1):
    next_start_time = time_to_seconds(combined_segments[i + 1]['start'])
    combined_segments[i]['end'] = format_time(round(next_start_time, 2))
    



print(type(combined_segments))
print(len(combined_segments))
print(combined_segments)
# for segment in combined_segments:
#     print(segment)


<class 'list'>
51
[{'text': "hi everyone so we are we are here to present the pp for the logon so without any further Ado let's start with the presentation so the problem statement a", 'start': '00:2.44', 'end': '00:16.12'}, {'text': 'problem statement is a self-learning a power PDF to data converter uh our team name is team conference the members of our team are M', 'start': '00:16.12', 'end': '00:25.68'}, {'text': 'myself ban and PR we are from vishak Institute of Technology Pune so the idea approach and details so a solution a proposed solution utilizes', 'start': '00:25.68', 'end': '00:38.84'}, {'text': 'a fine-tune multimodel that is lava which is capable of vision capabilities so it accepts uh image and text as a input and machine learning to extract', 'start': '00:38.84', 'end': '00:50.92'}, {'text': 'data from complex documents so these documents can be financial documents uh Supply Chain management reports or Healthcare reports any form of', 'start': '00:50.92', 'end': '01:1.2

djif

In [84]:
import nltk
nltk.download('punkt')

def generate_chunks(response, time_format="mm:ss:hh"):
  
  text_chunks = []
  for element in response:
    text = element['text']
    start_time = element['start']
    end_time = element['end']


    # Split text into sentences (optional)
    sentences = nltk.sent_tokenize(text)

    for sentence in sentences:
      text_chunk = f"{start_time} <= : {sentence} : => {end_time}"
      text_chunks.append(text_chunk)

  return text_chunks

# Assuming your API response is stored in a variable called 'response'
chunks = generate_chunks(combined_segments)

# Write the chunks to a file
with open("rag.txt", "w") as file:
  file.write("\n".join(chunks))



print("Chunks generated and written to rag.txt!")


Chunks generated and written to rag.txt!


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\auth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [85]:

# import
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
loader = TextLoader("./rag.txt")
documents = loader.load()

documents[0]

Document(page_content="00:2.44 <= : hi everyone so we are we are here to present the pp for the logon so without any further Ado let's start with the presentation so the problem statement a : => 00:16.12\n00:16.12 <= : problem statement is a self-learning a power PDF to data converter uh our team name is team conference the members of our team are M : => 00:25.68\n00:25.68 <= : myself ban and PR we are from vishak Institute of Technology Pune so the idea approach and details so a solution a proposed solution utilizes : => 00:38.84\n00:38.84 <= : a fine-tune multimodel that is lava which is capable of vision capabilities so it accepts uh image and text as a input and machine learning to extract : => 00:50.92\n00:50.92 <= : data from complex documents so these documents can be financial documents uh Supply Chain management reports or Healthcare reports any form of : => 01:1.24\n01:1.24 <= : structured PDF or any form of PDF document so our solution leverages generative a AI uh techniques

In [86]:

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1,
    chunk_overlap  = 0,
    length_function = lambda x: 1, # hack - usually len is used 
    is_separator_regex = False
)
split_docs = text_splitter.split_documents(documents)


len(split_docs) 

51

In [87]:
# split_docs[:4]

[Document(page_content="00:2.44 <= : hi everyone so we are we are here to present the pp for the logon so without any further Ado let's start with the presentation so the problem statement a : => 00:16.12", metadata={'source': './rag.txt'}),
 Document(page_content='00:16.12 <= : problem statement is a self-learning a power PDF to data converter uh our team name is team conference the members of our team are M : => 00:25.68', metadata={'source': './rag.txt'}),
 Document(page_content='00:25.68 <= : myself ban and PR we are from vishak Institute of Technology Pune so the idea approach and details so a solution a proposed solution utilizes : => 00:38.84', metadata={'source': './rag.txt'}),
 Document(page_content='00:38.84 <= : a fine-tune multimodel that is lava which is capable of vision capabilities so it accepts uh image and text as a input and machine learning to extract : => 00:50.92', metadata={'source': './rag.txt'})]

In [88]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(split_docs, embedding_function)

In [127]:
query = "Streamlit is used for ?"
rel_docs = db.similarity_search(query)


# print results
print(len(rel_docs))
print(rel_docs)

4
[Document(page_content="08:18.04 <= : using stream lit so it will provide us a easy interface and scalable as scalable as well user friendly approach for the user so we'll be using PDF minor so : => 08:29.64", metadata={'source': './rag.txt'}), Document(page_content="08:18.04 <= : using stream lit so it will provide us a easy interface and scalable as scalable as well user friendly approach for the user so we'll be using PDF minor so : => 08:29.64", metadata={'source': './rag.txt'}), Document(page_content='06:25.36 <= : so in finance it is invoice processing automat IC Ally extract vendor info and item details that might be in the invoices uh to get prices for a stream : => 06:37.80', metadata={'source': './rag.txt'}), Document(page_content='06:25.36 <= : so in finance it is invoice processing automat IC Ally extract vendor info and item details that might be in the invoices uh to get prices for a stream : => 06:37.80', metadata={'source': './rag.txt'})]


In [128]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")



prompt

ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based only on the provided context. \nThink step by step before providing a detailed answer. \nI will tip you $1000 if the user finds the answer helpful. \n<context>\n{context}\n</context>\nQuestion: {input}'))])

In [129]:
# rel_docs[0]

Document(page_content="08:18.04 <= : using stream lit so it will provide us a easy interface and scalable as scalable as well user friendly approach for the user so we'll be using PDF minor so : => 08:29.64", metadata={'source': './rag.txt'})

In [136]:
context = ""
for doc in rel_docs:
    context += doc.page_content + "\n"  

question = "What is Streamlit?"



print(context)
print(question)


08:18.04 <= : using stream lit so it will provide us a easy interface and scalable as scalable as well user friendly approach for the user so we'll be using PDF minor so : => 08:29.64
08:18.04 <= : using stream lit so it will provide us a easy interface and scalable as scalable as well user friendly approach for the user so we'll be using PDF minor so : => 08:29.64
06:25.36 <= : so in finance it is invoice processing automat IC Ally extract vendor info and item details that might be in the invoices uh to get prices for a stream : => 06:37.80
06:25.36 <= : so in finance it is invoice processing automat IC Ally extract vendor info and item details that might be in the invoices uh to get prices for a stream : => 06:37.80

What is Streamlit?


In [139]:
prompt_formatted = prompt.format(
    input=question,
    context = context,
)



print(prompt_formatted)

Human: 
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
08:18.04 <= : using stream lit so it will provide us a easy interface and scalable as scalable as well user friendly approach for the user so we'll be using PDF minor so : => 08:29.64
08:18.04 <= : using stream lit so it will provide us a easy interface and scalable as scalable as well user friendly approach for the user so we'll be using PDF minor so : => 08:29.64
06:25.36 <= : so in finance it is invoice processing automat IC Ally extract vendor info and item details that might be in the invoices uh to get prices for a stream : => 06:37.80
06:25.36 <= : so in finance it is invoice processing automat IC Ally extract vendor info and item details that might be in the invoices uh to get prices for a stream : => 06:37.80

</context>
Question: What is Streamlit?
